In [1]:
from haversine import haversine_vector
import pandas as pd
import numpy as np
import glob
import os
import sys
sys.path.append('../resources/library')
from tropical_cyclone.georeferencing import round_to_grid
from tropical_cyclone.cyclone import init_track_dataframe, tracking_algorithm, paper_tracking_algorithm, track_matching
from tropical_cyclone.visualize import plot_tracks, plot_detections
from tropical_cyclone.macros import TEST_YEARS as test_years

import warnings
warnings.filterwarnings('ignore')

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-cbvoltgo because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
/opt/conda/lib/python3.10/site-packages/torch/__config__.py:10: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._show_config()


In [2]:
# define inference directory to draw detections
dataset_dir = '../data/inference'
available_models = sorted([folder for folder in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, folder))])
available_models

['graphunet']

In [3]:
# select the model to analyze
#selected_model = '11_swin_fg10_t_500_msl_vo_850'
selected_model = 'graphunet'

# get ibtracs directory
ibtracs_src = '../data/ibtracs/filtered/ibtracs_main-tracks_6h_1980-2021_TS-NR-ET-MX-SS-DS.csv'

# define test years (same as paper)
#test_years = [i for i in range(1980,2020)]
# test_years = [1983, 1984, 1993, 1994, 2003, 2004, 2013, 2014]
# test_years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
# test_years = [1993]

# kilometer threshold
max_distance_detection = 1000.0

# whether or not to make plots
plot = False

In [4]:
# get model directory
model_dir = os.path.join(dataset_dir, selected_model)
# get inference filenames
inference_files = [os.path.join(model_dir, f'{year}.csv') for year in test_years]
model_dir, inference_files

('../data/inference/graphunet', ['../data/inference/graphunet/1997.csv'])

In [5]:
# load csv files
csv_files = []
for file in inference_files:
    csv_files.append(pd.read_csv(file, index_col=0))
# merge csv files together
detections = pd.concat(csv_files).reset_index(drop=True)
# convert iso time with pandas
detections['ISO_TIME'] = pd.to_datetime(detections['ISO_TIME'])
# add WS as np.inf
detections['WS'] = np.inf
detections

,ISO_TIME,LAT,LON,WS
0,1997-01-11,61.00,221.25,inf
1,1997-01-11,65.00,237.00,inf
2,1997-01-11,51.50,219.50,inf
3,1997-01-11,56.75,223.25,inf
4,1997-01-11,40.75,207.00,inf
...,...,...,...,...
91096,1997-12-23,5.00,279.75,inf
91097,1997-12-23,7.50,287.50,inf
91098,1997-12-23,8.25,296.25,inf
91099,1997-12-23,8.50,309.75,inf


In [6]:
columns = ['ISO_TIME','SID','NATURE','WMO_WIND','LAT','LON']
# load ibtracs
observations = pd.read_csv(ibtracs_src, index_col=0)
# convert iso time with pandas
observations['ISO_TIME'] = pd.to_datetime(observations['ISO_TIME'])
# get only some columns from ibtracs
observations = observations[columns]
# round lat and lon to be comparable with training data
observations['LAT'] = round_to_grid(observations['LAT'], grid_res=0.25)
observations['LON'] = round_to_grid(observations['LON'], grid_res=0.25)
observations

,ISO_TIME,SID,NATURE,WMO_WIND,LAT,LON
0,1980-03-16 00:00:00,1980076N06148,NR,,6.00,147.75
1,1980-03-16 06:00:00,1980076N06148,NR,,6.00,147.25
2,1980-03-16 12:00:00,1980076N06148,NR,,6.00,146.75
3,1980-03-16 18:00:00,1980076N06148,NR,,6.00,146.50
4,1980-03-17 00:00:00,1980076N06148,NR,,6.00,146.00
...,...,...,...,...,...,...
86188,2021-12-16 06:00:00,2021349N05108,TS,,4.50,105.00
86189,2021-12-16 12:00:00,2021349N05108,TS,,4.25,104.50
86190,2021-12-16 18:00:00,2021349N05108,TS,,4.25,103.00
86191,2021-12-17 00:00:00,2021349N05108,TS,,4.25,103.00


In [7]:
tmp = pd.merge(left=detections, right=observations, on='ISO_TIME', how='inner')
tmp = tmp[tmp['ISO_TIME'].dt.year.isin(test_years)]
dates = tmp['ISO_TIME'].to_numpy()

In [8]:
print(f'There are {len(observations)} observations and {len(detections)} detections')

There are 86193 observations and 91101 detections


In [9]:
# get only detections and observations present on both dataframes
detections = detections[detections['ISO_TIME'].isin(dates)].reset_index(drop=True)
observations = observations[observations['ISO_TIME'].isin(dates)].reset_index(drop=True)

In [10]:
print(f'There are {len(observations)} observations and {len(detections)} detections')

There are 2397 observations and 91003 detections


# Localization

In [11]:
# merge together detections and ibtracs
matches = pd.merge(left=detections, right=observations, on='ISO_TIME')
# compute haversine distance between any couple of points
matches['HDIST'] = haversine_vector(array1=matches[['LAT_x','LON_x']].to_numpy(), array2=matches[['LAT_y','LON_y']].to_numpy(), normalize=True)
matches.head()

,ISO_TIME,LAT_x,LON_x,WS,SID,NATURE,WMO_WIND,LAT_y,LON_y,HDIST
0,1997-01-11,61.00,221.25,inf,1997011N04176,NR,,4.0,176.0,7375.404151
1,1997-01-11,65.00,237.00,inf,1997011N04176,NR,,4.0,176.0,8281.568869
2,1997-01-11,51.50,219.50,inf,1997011N04176,NR,,4.0,176.0,6634.504419
3,1997-01-11,56.75,223.25,inf,1997011N04176,NR,,4.0,176.0,7178.430306
4,1997-01-11,40.75,207.00,inf,1997011N04176,NR,,4.0,176.0,5126.884366


In [12]:
# remove all the distances above `max_distance_localization` km
matches = matches[matches['HDIST'] < max_distance_detection]
# group by LATx and LONx and find the minimum (to remove x duplicates)
matches = matches.groupby(by=['ISO_TIME','LAT_x','LON_x','SID','NATURE','WMO_WIND']).min('HDIST').reset_index()
# repeat grouping by LATy and LONy and find the minimum (to remove y duplicates)
matches = matches.groupby(by=['ISO_TIME','LAT_y','LON_y','SID','NATURE','WMO_WIND']).min('HDIST').reset_index()
# show result
matches.head()

,ISO_TIME,LAT_y,LON_y,SID,NATURE,WMO_WIND,LAT_x,LON_x,WS,HDIST
0,1997-01-11 00:00:00,4.0,176.00,1997011N04176,NR,,8.00,168.50,inf,715.699544
1,1997-01-11 06:00:00,4.0,175.50,1997011N04176,NR,,6.25,167.25,inf,647.216313
2,1997-01-11 12:00:00,4.0,174.75,1997011N04176,NR,,5.75,176.75,inf,692.625160
3,1997-01-11 18:00:00,4.0,174.00,1997011N04176,NR,,6.25,175.75,inf,641.353435
4,1997-01-12 00:00:00,4.0,173.25,1997011N04176,NR,,7.00,176.75,inf,794.958950


In [13]:
min_distance_localization = matches['HDIST'].min()
max_distance_localization = matches['HDIST'].max()
mean_distance_localization = matches['HDIST'].mean()
median_distance_localization = matches['HDIST'].median()

print(f"Model {selected_model} Localization results")
print(f"   Min distance ({np.round(min_distance_localization,2)} km)")
print(f"   Max distance ({np.round(max_distance_localization,2)} km)")
print(f"   Average distance ({np.round(mean_distance_localization,2)} km)")
print(f"   Median distance ({np.round(median_distance_localization,2)} km)")

Model graphunet Localization results
   Min distance (0.0 km)
   Max distance (955.2 km)
   Average distance (201.68 km)
   Median distance (138.99 km)


In [14]:
if plot: plot_detections(detections, observations)

# Classification

In [15]:
def F_beta(beta, precision, recall):
    return (1 + beta**2) * ((precision * recall) / ((beta**2 * precision) + recall))

In [16]:
n_dets = len(detections)
n_tp = len(matches)
n_obs = len(observations)
n_fp = n_dets - n_tp
n_fn = n_obs - n_tp

In [17]:
precision = n_tp / (n_tp + n_fp)
recall = n_tp / (n_tp + n_fn)
f2_score = F_beta(beta=2, precision=precision, recall=recall) * 100

In [18]:
print(f"Model {selected_model} Classification results")
print(f"   F2 : {np.round(f2_score,2)} % (precision={np.round(precision, 2)}, recall={np.round(recall,2)})")
print(f"   TP : {n_tp} out of {n_obs} observations ({np.round(n_tp / n_obs * 100)} %)")
print(f"   FP : {n_fp} out of {n_dets} ML detections ({np.round(n_fp / n_dets * 100)} %)")
print(f"   FN : {n_fn} out of {n_obs} observations ({np.round(n_fn / n_obs * 100)} %)")

Model graphunet Classification results
   F2 : 11.83 % (precision=0.03, recall=0.99)
   TP : 2379 out of 2397 observations (99.0 %)
   FP : 88624 out of 91003 ML detections (97.0 %)
   FN : 18 out of 2397 observations (1.0 %)


# Tracking

In [19]:
# minimum track length (1 day)
min_track_count = 12
# minimum speed of wind in order to consider the track true
min_wind_speed = 17.0
# maximum distance between tracks
max_track_distance_tracking = 400.0

grid_res = 0.25
km_to_deg = 110.474

In [20]:
# rename SID to TRACK_ID
observed_tracks = observations.rename(columns={'SID':'TRACK_ID'})
# get only long enough tracks for the comparison
valid_observations_sids = observed_tracks.groupby('TRACK_ID').filter(lambda x: len(x) >= min_track_count)['TRACK_ID'].unique()
# filter out the observations
observed_tracks = observed_tracks[observed_tracks['TRACK_ID'].isin(valid_observations_sids)].reset_index(drop=True)
observed_tracks.head()

,ISO_TIME,TRACK_ID,NATURE,WMO_WIND,LAT,LON
0,1997-01-11 00:00:00,1997011N04176,NR,,4.0,176.00
1,1997-01-11 06:00:00,1997011N04176,NR,,4.0,175.50
2,1997-01-11 12:00:00,1997011N04176,NR,,4.0,174.75
3,1997-01-11 18:00:00,1997011N04176,NR,,4.0,174.00
4,1997-01-12 00:00:00,1997011N04176,NR,,4.0,173.25


In [21]:
# apply tracking scheme
tracking_src = f'/ceph/hpc/home/ciangottinid/ml-tropical-cyclones-detection/data/inference/{selected_model}/tracking.csv'
if not os.path.exists(tracking_src):
    detected_tracks = init_track_dataframe(detections)
    detected_tracks = tracking_algorithm(detected_tracks, max_track_distance_tracking, min_track_count)
    # detected_tracks = paper_tracking_algorithm(detected_tracks, max_track_distance_tracking, min_track_count)
    detected_tracks.to_csv(tracking_src)
else:
    detected_tracks = pd.read_csv(tracking_src, index_col=0)
detected_tracks.head()

# detected_tracks = init_track_dataframe(detections)
# detected_tracks = tracking_algorithm(detected_tracks, max_track_distance_tracking, min_track_count)
# detected_tracks = paper_tracking_algorithm(detected_tracks, max_track_distance_tracking, min_track_count)

,ISO_TIME,LAT,LON,WS,TRACK_ID,HAVERSINE
0,1997-01-11 00:00:00,10.25,108.75,inf,1997011100_48,109.420440
1,1997-01-11 00:00:00,20.00,116.50,inf,1997011100_49,370.387777
2,1997-01-11 00:00:00,18.50,120.00,inf,1997011100_50,0.000000
3,1997-01-11 00:00:00,11.00,240.00,inf,1997011100_62,0.000000
4,1997-01-11 00:00:00,10.00,318.25,inf,1997011100_91,215.959982


In [22]:
print(f'There are:')
print(f'   - {len(detected_tracks["TRACK_ID"].unique())} detected tracks')
# print(f'   - {len(paper_detected_tracks["TRACK_ID"].unique())} detected tracks (paper)')
print(f'   - {len(observed_tracks["TRACK_ID"].unique())} observed tracks')

There are:
   - 841 detected tracks
   - 56 observed tracks


In [23]:
if plot:plot_tracks(detected_tracks, observed_tracks)
# plot_tracks(detected_tracks[pd.to_datetime(detected_tracks['ISO_TIME']).dt.year.isin([1993])], observed_tracks[pd.to_datetime(observed_tracks['ISO_TIME']).dt.year.isin([1993])])

# Track Matching

In [ ]:
import dynamicopy

# maximum distance to consider true the match
max_track_distance_matching = 300.0

Failure in importing the cartopy library, the dynamicopy.cartoplot will not be loaded.     Please install cartopy if you wish to use it.
Failure in importing the cartopy library, the dynamicopy.cartoplot will not be loaded.     Please install cartopy if you wish to use it.


##### Our track matching algorithm provides same results of Bourdin's (but more slowly)

In [25]:
# track_matches = track_matching(detected_tracks, observed_tracks, max_track_distance_matching)
# # H = HITS = True Positive
# H = len(track_matches[(track_matches['DET_TRACK_ID']!='') & (track_matches['OBS_TRACK_ID']!='')])
# # M = Miss = False Negative
# M = len(track_matches[(track_matches['DET_TRACK_ID']=='') & (track_matches['OBS_TRACK_ID']!='')])
# # FA = False Alarm = False Positive
# FA = len(track_matches[(track_matches['DET_TRACK_ID']!='') & (track_matches['OBS_TRACK_ID']=='')])

# POD = (H / (H + M))
# FAR = (FA / (H + FA))

# print(f"Hits : {H}")
# print(f"Miss : {M}")
# print(f"False Alarm : {FA}")
# print(f"POD : {POD}")
# print(f"FAR : {FAR}")

In [26]:
# # load bourdin observed tracks from library
# bourdin_observed_tracks = pd.read_csv('/Users/davide/Developer/ml-tropical-cyclones-detection/resources/library/zenodo_bourdin/ibtracs/ibtracs.since1980.cleaned.csv', index_col=0)

# # convert columns to dynamicopy compliant format
# detected_tracks = detected_tracks.rename(columns={'ISO_TIME':'time','LAT':'lat','LON':'lon','TRACK_ID':'track_id'})
# observed_tracks = observed_tracks.rename(columns={'ISO_TIME':'time','LAT':'lat','LON':'lon','TRACK_ID':'track_id'})

# # convert time column pandas datetime format
# detected_tracks['time'] = pd.to_datetime(detected_tracks['time'])
# observed_tracks['time'] = pd.to_datetime(observed_tracks['time'])
# bourdin_observed_tracks['time'] = pd.to_datetime(bourdin_observed_tracks['time'])

# # convert longitudes to range [0, 360] format
# detected_tracks['lon'] = (detected_tracks['lon'] + 540) % 360 - 180
# observed_tracks['lon'] = (observed_tracks['lon'] + 540) % 360 - 180
# bourdin_observed_tracks['lon'] = (bourdin_observed_tracks['lon'] + 540) % 360 - 180

# # remove out of bound (both space and time) detections from bourdin observations
# bourdin_observed_tracks = bourdin_observed_tracks[(bourdin_observed_tracks['lon']>=100) & (bourdin_observed_tracks['lon']<=320) & (bourdin_observed_tracks['lat']>=0) & (bourdin_observed_tracks['lat']<=70)]
# bourdin_observed_tracks = bourdin_observed_tracks[bourdin_observed_tracks['time'].isin(dates)]
# bourdin_observed_tracks = bourdin_observed_tracks[bourdin_observed_tracks['track_id'].isin(bourdin_observed_tracks.groupby('track_id').filter(lambda x: len(x) >= min_track_count)['track_id'].unique())].reset_index(drop=True)

In [ ]:
# bourdin_track_matches = dynamicopy.tc.match_tracks(detected_tracks, bourdin_observed_tracks, "ours", 'bourdin', max_dist=max_track_distance_matching, min_overlap=0, ref=True)

# n_ib_match = len(bourdin_track_matches[f'id_bourdin'].unique())
# n_our_match = len(bourdin_track_matches['id_ours'].unique())
# n_observations = len(bourdin_observed_tracks.track_id.unique())
# n_detections = len(detected_tracks.track_id.unique())

# H, M, FA = n_ib_match, n_observations - n_ib_match, n_detections - n_our_match
# POD = H / (H + M)
# FAR = FA / (H + FA)

# print(f"Hits : {H}")
# print(f"Misses : {M}")
# print(f"False Alarms : {FA}")
# print(f"POD : {POD}")
# print(f"FAR : {FAR}")

In [28]:
# fix for upper/lowercase problems with column names
detected_tracks.columns = detected_tracks.columns.str.lower()
observed_tracks.columns = observed_tracks.columns.str.lower()

# fix for 'iso_time' being required as 'time' by dynamicopy
detected_tracks.rename(columns={'iso_time': 'time'}, inplace=True)
observed_tracks.rename(columns={'iso_time': 'time'}, inplace=True)

# fix for longitude out of range [-180, 180]
detected_tracks.lon -= 180
observed_tracks.lon -= 180

detected_tracks.columns, observed_tracks.columns

(Index(['time', 'lat', 'lon', 'ws', 'track_id', 'haversine'], dtype='object'),
 Index(['time', 'track_id', 'nature', 'wmo_wind', 'lat', 'lon'], dtype='object'))

In [ ]:
# ensuring compatibility between different Pandas versions
detected_tracks['time'] = pd.to_datetime(detected_tracks['time'])
observed_tracks['time'] = pd.to_datetime(observed_tracks['time'])

track_matches = dynamicopy.tc.match_tracks(detected_tracks, observed_tracks, "ours", 'ibtracs', max_dist=max_track_distance_matching, min_overlap=0, ref=True)

n_ib_match = len(track_matches[f'id_ibtracs'].unique())
n_our_match = len(track_matches['id_ours'].unique())
n_observations = len(observed_tracks.track_id.unique())
n_detections = len(detected_tracks.track_id.unique())

H, M, FA = n_ib_match, n_observations - n_ib_match, n_detections - n_our_match
POD = H / (H + M)
FAR = FA / (H + FA)

# POD, FAR, H, M, FA
print(f"Hits : {H}")
print(f"Misses : {M}")
print(f"False Alarms : {FA}")
print(f"POD : {POD}")
print(f"FAR : {FAR}")

Hits : 52
Misses : 4
False Alarms : 690
POD : 0.9285714285714286
FAR : 0.9299191374663073


# Save to file

In [30]:
columns = [
    'model', 'max_distance_detection', 'n_dets', 'n_tp', 'n_obs', 'n_fp', 'n_fn', 'precision', 
    'recall', 'f2_score', 'min_distance_localization', 'max_distance_localization', 
    'mean_distance_localization', 'median_distance_localization', 'min_track_count', 
    'max_distance_tracking', 'min_wind_speed', 'max_track_distance_matching', 
    'max_track_distance_tracking', 'H', 'M', 'FA', 'POD', 'FAR', 'ibtracs_src', 'test_years', 
]
dst = f'/ceph/hpc/home/ciangottinid/ml-tropical-cyclones-detection/data/inference/{selected_model}/results_analysis.csv'

if os.path.exists(dst):
    results = pd.read_csv(dst, index_col=0)
else:
    results = pd.DataFrame(columns=columns)

results = pd.concat([results, pd.DataFrame(data={
    'model': [selected_model], 'max_distance_detection': [max_distance_detection], 'n_dets': [n_dets], 'n_tp': [n_tp], 
    'n_obs': [n_obs], 'n_fp': [n_fp], 'n_fn': [n_fn], 'precision': [precision], 'recall': [recall], 
    'f2_score': [f2_score], 'min_distance_localization': [min_distance_localization], 'max_distance_localization': [max_distance_localization], 
    'mean_distance_localization': [mean_distance_localization], 'median_distance_localization': [median_distance_localization], 
    'min_track_count': [min_track_count], 'max_distance_tracking': [max_track_distance_tracking], 'min_wind_speed': [min_wind_speed], 
    'max_track_distance_matching': [max_track_distance_matching], 'max_track_distance_tracking': [max_track_distance_tracking], 
    'H': [H], 'M': [M], 'FA': [FA], 'POD': [POD], 'FAR': [FAR], 'ibtracs_src': [ibtracs_src], 'test_years': [test_years], 
})])

results = results.reset_index(drop=True)
results.to_csv(dst)

# Paper Results

In the paper, with the ML ensemble we have the following results:

- F2-score : 53 %
- Euclidean distance : 117.06 km
- Hit rate : 88.91 %
- POD : 71.49 %
- FAR : 23.00 %